In [78]:
import sys
sys.path.append("../")

In [79]:
from utils.common_train_utils import *
from utils.draw_candle_image import *
from utils.evaluate_old_models import *
from functools import partial
import os
from tabulate import tabulate
import random
import warnings
warnings.filterwarnings("ignore")


In [80]:
def print_loss_of_all_models_in_dataset_folder(folder_object):
    dataset_path = folder_object.path
    
    config = read_config(dataset_path)
    candle_type_and_directory_save: dict = config["candle_type_and_directory_save"]
    days_result = config["days_result"]
    
    dataset_test = load_dataset_of_each_type_and_combine(f"{dataset_path}/test", candle_type_and_directory_save)
    
    batch_size = random.randint(1, int(len(dataset_test) / 4))
    shuffle_buffer = random.randint(1, int(len(dataset_test) / 3))
    
    dataset_test_1 = dataset_test.shuffle(shuffle_buffer)
    dataset_test_2 = dataset_test_1.batch(batch_size)
    
    # Ví dụ
    all_models_folder_path = os.path.join(os.path.abspath(dataset_path), "model_save")
    df = evaluate_all_models_in_dataset_folder(
        all_models_folder_path, 
        dataset_test_2,
        days_result
    )
    
    # print(f"----------------- {folder_name} -----------------")
    # print(df)
    
    df["total_records"] = len(dataset_test)
    df["batch_size"] = batch_size
    df["shuffle_buffer"] = shuffle_buffer
    
    return df

In [81]:
dataset_parent_folder = os.path.abspath("../dataset")

In [82]:
all_dfs = {}
for f in os.scandir(dataset_parent_folder):
    if not f.is_dir(): continue
    
    df = print_loss_of_all_models_in_dataset_folder(f)
    all_dfs[f.name] = df

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 1.5200 - mae: 0.9211 - mse: 1.5200
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 1.4290 - mae: 0.8917 - mse: 1.4290
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 1.4271 - mae: 0.8876 - mse: 1.4271
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 1.6120 - mae: 0.9423 - mse: 1.6120
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 1.3765 - mae: 0.8697 - mse: 1.3765
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 3.6454 - mae: 1.3431 - mse: 3.6454
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - loss: 4.0551 - mae: 1.3255 - mse: 4.0551
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - loss: 4.1828 - mae: 1.3275 - mse: 4.1828
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 4.0412 - mae: 1.2829 - mse: 4.0412
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - loss: 4.0865 - mae: 1.2887 - mse: 4.0865
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - loss: 3.9516 - mae: 1.2692 - mse: 3.9516
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - loss: 4.0429 - mae: 1.2911 - mse: 4.0429
5/5 ━━━━━━━━

In [83]:
list(all_dfs.keys())

['apple_with_ema_macd_trend',
 'btc_with_ema_macd_trend',
 'btc_with_ema_macd_trend_old',
 'coca_cola_with_ema_macd_trend',
 'ctg_with_ema_macd_trend',
 'eth_with_ema_macd_trend',
 'fpt_with_ema_macd_trend',
 'tcb_with_ema_macd_trend',
 'vn30_with_ema_macd_trend']

In [84]:
for dataset_name in all_dfs:
    print(dataset_name)
    print(tabulate(all_dfs[dataset_name], headers='keys', tablefmt='pretty'))
    print("\n")

apple_with_ema_macd_trend
+---+---------------------------------------+--------------------+--------------------+---------------+------------+----------------+
|   |              model_name               |        mse         |        mae         | total_records | batch_size | shuffle_buffer |
+---+---------------------------------------+--------------------+--------------------+---------------+------------+----------------+
| 1 |    model_5_biLSTM_ema_macd_trend_1    | 1.3436095714569092 | 0.8511976003646851 |      103      |     7      |       20       |
| 2 |   model_5_biLSTM_ema_macd_trend_1_1   | 1.3958796262741089 | 0.8672536015510559 |      103      |     7      |       20       |
| 0 |     model_5_biLSTM_ema_macd_trend     | 1.3958799839019775 | 0.8672536611557007 |      103      |     7      |       20       |
| 4 | model_5_biLSTM_ema_macd_trend_1_1_1_1 | 1.3958989381790161 | 0.8672159314155579 |      103      |     7      |       20       |
| 3 |  model_5_biLSTM_ema_macd_trend